In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from utils import *
from siamese import *
from visualize_attributes import *

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
root_path = os.path.dirname(os.getcwd())
data_path = os.path.join(root_path, "data/multi_classed_grouped_data")
products = os.listdir(data_path)
train = True
visualize = True
#products = ['bottle', 'hazelnut', 'metal_nut', 'toothbrush', 'transistor', 'leather', 'cable',  'zipper', 'wood']
print(f"Using {device}")

transform = transforms.Compose([transforms.ToTensor(),
                            transforms.Resize((1024,1024), antialias=True)
                            ])

for backbone in ['resnet18', 'resnet34', 'resnet50', 'wide_resnet', 'vgg']:
    print('---------------------------------------------- ' + backbone + ' ----------------------------------------------')
    for product in products:
        print('----------------------------------- ' + product + ' -----------------------------------')
        model_path = os.path.join(root_path, 'models', product + '_siamese_' + backbone + '_subclass_sampling.pth')
        if not os.path.exists(model_path):
            print('No model for this backbone')
            continue
        if backbone in ['wide_resnet', 'vgg11']:
            device = 'cpu'

        # Load data dicts
        product_path = os.path.join(data_path, product)
        json_path = os.path.join(root_path, 'data/augmented_train_test_split', product)
        train_dict = json.load(open(os.path.join(json_path, product + '_train_dict.json'), 'r'))
        test_dict = json.load(open(os.path.join(json_path, product + '_test_dict.json'), 'r'))

        # Load golden image
        golden_image_path = os. path.join(root_path, "data/golden_images", product)
        golden_image_path = os.path.join(golden_image_path, os.listdir(golden_image_path)[0])
        
        model_image_dict = {golden_image_path : 1}
        model_dataset = ProductDataset(model_image_dict, transform=transform)
        model_dataloader = DataLoader(model_dataset, batch_size=1, shuffle=True)

        model_image, _, model_label, _ = next(iter(model_dataloader))
        model_label = model_label[0, 0]
        
        # Load model
        n_classes = len(np.unique(list(train_dict.values())))
        model = SiameseNetwork(n_classes, backbone=backbone)
        
        model.load_state_dict(torch.load(model_path))
        model.eval()
        
        all_score = []
        i = 0
        for image_path in tqdm(list(test_dict.keys())):
            image_dict = {image_path: test_dict[image_path]}
            
            test_dataset = ProductDataset(image_dict, transform=transform)
            test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
            test_image, _, test_label, _ = next(iter(test_dataloader))
            test_label = test_label[0, 0]
            
            all_distance = []
            model.to(device)
            
            # print(model_label, test_label)
            output_1, output_2 = model(model_image.to(device), test_image.to(device))
            euclidean_distance = F.pairwise_distance(output_1, output_2, keepdim=True).item()
            
            all_score.append(np.array([1, int(test_label), 1 == int(test_label), euclidean_distance]))
            # print(avg_distance)

            if i % 5 == 0:
                model_image_np = np.array(model_image[0].permute(1, 2, 0)*255, dtype=np.uint8)
                test_image = np.array(test_image[0].permute(1, 2, 0)*255, dtype=np.uint8)
                label_1 = "good"
                label_2 = "defect" if test_label.numpy() == 0 else "good"
                label = "same class" if label_1 == label_2 else "different class"
                fig, ax = plt.subplots(1, 2, figsize=(10, 5))
                ax[0].imshow(model_image_np[:,:,::-1])
                ax[0].set_xlabel(label_1, weight = 'bold', fontsize = 20)
                #ax[0].axis('off')
                ax[1].imshow(test_image[:,:,::-1])
                ax[1].set_xlabel(label_2, weight = 'bold', fontsize = 20)
                #ax[1].axis('off')
                fig.suptitle('Dissimilarity score: ' + str(euclidean_distance))
                figure_name = product + '_siamese_' + backbone + '_visualize_' + str(i) + '_golden_image.png'
                figure_path = os.path.join(root_path, 
                        'Figure/visualize_inference/golden_image', product)
                if not os.path.exists(figure_path):
                    os.mkdir(figure_path)
                figure_path = os.path.join(figure_path, figure_name)
                fig.savefig(figure_path)
            i += 1
        result = np.array(all_score)
        score_path = os.path.join(root_path, 'result/similarity_scores', product + '_siamese_' + backbone + '_golden_image.csv')
        np.savetxt(score_path, result, delimiter=",")
        
        same_class = result[:,2]
        if len(np.unique(same_class)) != 2:
            print(np.unique(same_class))
        similarity_scores = result[:,3]
        
        dissimillar_idx = np.where(same_class == 0.)
        dissimillar_score = similarity_scores[dissimillar_idx]
        simillar_idx = np.where(same_class == 1.)
        simillar_score = similarity_scores[simillar_idx]
        #print(dissimillar_idx, simillar_idx)
        #print(dissimillar_score, simillar_score)
        fig, ax = plt.subplots(1, 2, figsize=(16, 8))
        ax[0].hist(simillar_score, edgecolor='black',)
                   #bins=np.arange(simillar_score.min(), simillar_score.max(), 0.1))
        ax[0].set_xlabel('Distance', fontsize = 10)
        ax[0].set_ylabel('Number of instance', fontsize = 10)
        ax[0].set_title(product + ': Similarity score distribution for images for GOOD class')
        ax[1].hist(dissimillar_score, edgecolor='black',)
                   #bins=np.arange(dissimillar_score.min(),dissimillar_score.max(), 0.1))
        ax[1].set_xlabel('Distance', fontsize = 10)
        ax[1].set_ylabel('Number of instance', fontsize = 10)
        ax[1].set_title(product + ': Similarity score distribution for images for DEFECT class')
        fig.show()

        figure_name = product + '_siamese_' + backbone + '_similarity_score_distribution' + '_golden_image.png'
        figure_path = os.path.join(root_path, 
                'Figure/golden_image_similarity_score_histogram/' + figure_name)
        fig.savefig(figure_path)


        # fig, ax = plt.subplots(1, 1, figsize=(16, 8))
        # ax.hist(simillar_score, edgecolor='black', label = 'good')
        # ax.hist(dissimillar_score, alpha=0.4, edgecolor='black', label = 'defect')
        #            #bins=np.arange(dissimillar_score.min(),dissimillar_score.max(), 0.1))
        # ax.set_xlabel('Distance', fontsize = 10)
        # ax.set_ylabel('Number of instance', fontsize = 10)
        # ax.set_title(product + ': Similarity score distribution of both GOOD and DEFECT')
        # ax.legend()
        # figure_name = product + '_siamese_' + backbone + 'all_similarity_score_distribution' + '_embedded_scoring.png'
        # figure_path = os.path.join(root_path, 
        #         'Figure/embedded_similarity_score_histogram/' + figure_name)
        # fig.savefig(figure_path)

Using cuda
---------------------------------------------- resnet18 ----------------------------------------------
----------------------------------- bottle -----------------------------------


  0%|          | 0/109 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 9.75 GiB total capacity; 508.83 MiB already allocated; 64.75 MiB free; 520.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF